### **Quickstart**

In [1]:
import os 
from dotenv import load_dotenv
load_dotenv()

api_key = os.environ["OPENAI_API_KEY"]

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", api_key=api_key)

In [3]:
from langchain_core.messages import HumanMessage

llm.invoke([HumanMessage("Chào , tôi là kytranmoi")])


AIMessage(content='Chào bạn, kytranmoi! Bạn có muốn hỏi hay chia sẻ điều gì không?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 16, 'total_tokens': 36, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0ba0d124f1', 'finish_reason': 'stop', 'logprobs': None}, id='run-f6124c69-2ca4-4791-be13-56d6c6a6342b-0', usage_metadata={'input_tokens': 16, 'output_tokens': 20, 'total_tokens': 36, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [4]:
llm.invoke([HumanMessage("Tên tôi là gì ?")])

AIMessage(content='Xin lỗi, nhưng tôi không biết tên của bạn. Bạn có thể cho tôi biết tên của bạn được không?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 12, 'total_tokens': 35, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0ba0d124f1', 'finish_reason': 'stop', 'logprobs': None}, id='run-88b4e4e6-f689-4b78-b72a-186c6b0e7d1c-0', usage_metadata={'input_tokens': 12, 'output_tokens': 23, 'total_tokens': 35, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

- Thì như ví dụ ở trên chúng ta thấy nó không hiểu được lịch sử của cuộc trò chuyện.
    -   Điều này thật là tệ hại với một con chatbot trong nó không khác gì so với việc tự đi search google.
- Để giải quyết vấn đề này chúng ta cần truyền vô toàn bộ lịch sử của cuộc trò chuyện.

In [5]:
from langchain_core.messages import AIMessage

llm.invoke(
    [
        HumanMessage("Chào , tôi là kytranmoi"),
        AIMessage("Chào bạn, kytranmoi! Bạn có muốn hỏi hay chia sẻ điều gì không?"),
        HumanMessage("Tôi là ai ?"),
    ]
)

AIMessage(content='Bạn là kytranmoi! Tuy nhiên, tôi không có thông tin cụ thể về bạn. Nếu bạn muốn chia sẻ thêm về bản thân, tôi rất vui được lắng nghe!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 49, 'total_tokens': 87, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0ba0d124f1', 'finish_reason': 'stop', 'logprobs': None}, id='run-c0a89fac-95d8-4f7a-a3d9-7212252d73db-0', usage_metadata={'input_tokens': 49, 'output_tokens': 38, 'total_tokens': 87, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

- Như vậy nó đã có thể phản hồi dựa trên lịch sử cũ của cuộc trò truyện.
- Đây là ý tưởng cơ bản để hỗ trợ khả nawg=ng tương tác đàm thoại của Chatbot. Vậy làm thế nào để triển khai điều này tốt nhất?

### **Message persistence**

- LangGraph triển khai lớp duy trì tích hợp, khiến nó trở nên lý tưởng cho các ứng dụng trò chuyện hỗ trợ nhiều lượt trò chuyện.
- Việc gói mô hình trò chuyện của chúng tôi trong ứng dụng LangGraph tối giản cho phép chúng tôi tự động lưu lại lịch sử tin nhắn, giúp đơn giản hóa quá trình phát triền các ứng dụng nhiều bước.
- LangGraph đi kèm với một checkpointer trong bộ nhớ đơn giản, chúng tôi sử dụng bên dưới.

In [11]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

workflow = StateGraph(state_schema=MessagesState)

def call_model(state: MessagesState):
    response = llm.invoke(state['messages'])
    return {"messages": response}

workflow.add_edge(START, "llm")
workflow.add_node("llm", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)


- Bây giờ chúng ta cần tạo một configmà chúng ta truyền vào runnable mọi lúc. Cấu hình này chứa thông tin không phải là một phần của đầu vào trực tiếp, nhưng vẫn hữu ích. Trong trường hợp này, chúng ta muốn bao gồm một thread_id. Nó sẽ trông như thế này:

In [12]:
config = {"configurable": {"thread_id": "abc123"}}

- Điều này cho phép chúng tôi hỗ trợ nhiều luồng hội thoại với một ứng dụng duy nhất, một yêu cầu phổ biến khi ứng dụng của bạn có nhiều người dùng.

In [13]:
query = """
Chào , tôi là kytranmoi
"""
input_message = [HumanMessage(query)]
output = app.invoke({"messages":input_message}, config=config)
output['messages'][-1].pretty_print()

================================== Ai Message ==================================

Chào bạn, kytranmoi! Rất vui được gặp bạn. Bạn cần giúp gì hôm nay?


In [18]:
query = "Tên tôi là gì ?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Tên bạn là kytranmoi. Nếu bạn cần tôi giúp gì khác, hãy cho tôi biết nhé!


- Tuyệt! Chatbot của chúng tôi hiện ghi nhớ mọi thứ về chúng tôi. Nếu chúng tôi thay đổi cấu hình để tham chiếu đến một thread_id, chúng tôi có thể thấy rằng nó bắt đầu cuộc trò chuyện mới.

In [19]:
config = {"configurable": {"thread_id": "abc234"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Xin lỗi, nhưng tôi không biết tên của bạn. Bạn có muốn chia sẻ tên của mình không?


In [20]:
config = {"configurable": {"thread_id": "abc123"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Tên bạn là kytranmoi. Nếu bạn có câu hỏi nào khác hoặc cần trợ giúp, hãy cho tôi biết!


- Đây là cách chúng ta có thể hỗ trợ chatbot trò chuyện với nhiều người dùng!

**Mẹo**
- Để hỗ trợ bất đồng bộ, hãy cập nhật call_modelnút thành hàm bất đồng bộ và sử dụng .ainvoke khi gọi ứng dụng:

In [29]:
import asyncio

async def call_model(state: MessagesState):
    response = llm.invoke(state['messages'])
    return {"messages": response}

workflow = StateGraph(state_schema=MessagesState)

workflow.add_edge(START, "llm")
workflow.add_node("llm", action=call_model)

app = workflow.compile(checkpointer=MemorySaver())

while True:
    query = input("You: ")
    if query == "exit":
        break
    input_messages = [HumanMessage(query)]
    output = await app.ainvoke({"messages": input_messages}, config=config)
    # (output["messages"][-1].pretty_print())
    # stream output
    for message in output["messages"]:
        message.pretty_print()



================================ Human Message =================================

chào
================================== Ai Message ==================================

Chào bạn! Bạn cần giúp gì hôm nay?
================================ Human Message =================================

chào
================================== Ai Message ==================================

Chào bạn! Bạn cần giúp gì hôm nay?
================================ Human Message =================================

tôi là kỳ trần moi đẹp trai tài năng mỗi tội bị người yêu đá
================================== Ai Message ==================================

Chào Kỳ Trần! Rất vui được gặp bạn. Mặc dù bị người yêu đá là một điều không dễ chịu, nhưng hãy nhớ rằng mọi chuyện rồi sẽ qua. Bạn có thể chia sẻ thêm về cảm xúc của mình không? Hoặc nếu bạn cần lời khuyên để vượt qua giai đoạn này, mình rất sẵn lòng giúp!
================================ Human Message =================================

chào
======================

- Hiện tại, tất cả những gì chúng ta đã làm là thêm một lớp duy trì đơn giản xung quanh mô hình. Chúng ta có thể bắt đầu làm cho chatbot phức tạp hơn và được cá nhân hóa hơn bằng cách thêm vào một mẫu nhắc nhở.

### **Prompt template**

- Mẫu nhắc nhở giúp chuyển thông tin người dùng thô thành định dạng mà LLM có thể làm việc. Trong trường hợp này, đầu vào người dùng thô chỉ là một thông báo, mà chúng ta đang chuyển đến LLM. Bây giờ chúng ta hãy làm cho nó phức tạp hơn một chút. Đầu tiên, hãy thêm vào một thông báo hệ thống với một số hướng dẫn tùy chỉnh (nhưng vẫn lấy thông báo làm đầu vào). Tiếp theo, chúng ta sẽ thêm vào nhiều đầu vào hơn ngoài các thông báo.

- Để thêm vào một thông báo hệ thống, chúng ta sẽ tạo một ChatPromptTemplate. Chúng ta sẽ sử dụng MessagesPlaceholderđể truyền tất cả các thông báo vào.

In [42]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Giả sử bạn là người yêu cũ của tôi , hãy trả lời câu hỏi của tôi",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
    
)

In [43]:
workflow = StateGraph(state_schema=MessagesState)


async def call_model(state: MessagesState):
    chain = prompt | llm
    response = chain.invoke(state)
    return {"messages": response}


workflow.add_edge(START, "llm")
workflow.add_node("llm", action=call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [46]:
config = {"configurable": {"thread_id": "abc345"}}
query = "tao ghét mày"

input_messages = [HumanMessage(query)]
output = await app.ainvoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Mình hiểu cảm giác đó, và mình tôn trọng cảm xúc của bạn. Nếu bạn muốn, mình có thể lắng nghe lý do tại sao bạn cảm thấy như vậy. Bạn có muốn chia sẻ không?


### **Managing Conversation History**

- Một khái niệm quan trọng cần hiểu khi xây dựng chatbot là cách quản lý lịch sử trò chuyện. Nếu không được quản lý, danh sách tin nhắn sẽ tăng lên không giới hạn và có khả năng tràn cửa sổ ngữ cảnh của LLM. Do đó, điều quan trọng là phải thêm một bước giới hạn kích thước của tin nhắn bạn đang truyền vào.

- Điều quan trọng là bạn sẽ muốn thực hiện việc này TRƯỚC mẫu lời nhắc nhưng SAU khi bạn tải các tin nhắn trước đó từ Lịch sử tin nhắn.

- Chúng ta có thể thực hiện điều này bằng cách thêm một bước đơn giản trước dấu nhắc để sửa đổi messages khóa một cách thích hợp, sau đó gói chuỗi mới đó vào lớp Lịch sử tin nhắn.

- LangChain đi kèm một số trình trợ giúp tích hợp để quản lý danh sách tin nhắn . Trong trường hợp này, chúng ta sẽ sử dụng trình trợ giúp trim_messages để giảm số lượng tin nhắn chúng ta đang gửi đến mô hình. Trình cắt cho phép chúng ta chỉ định số lượng mã thông báo chúng ta muốn giữ lại, cùng với các tham số khác như chúng ta có muốn luôn giữ lại tin nhắn hệ thống hay không và có cho phép tin nhắn một phần hay không:

In [47]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=llm,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

- Để sử dụng nó trong chuỗi của chúng ta, chúng ta chỉ cần chạy trình cắt trước khi truyền messagesdữ liệu đầu vào đến dấu nhắc.

In [48]:
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict


class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str
    
workflow = StateGraph(state_schema=State)


def call_model(state: State):
    chain = prompt | llm
    trimmed_messages = trimmer.invoke(state["messages"])
    response = chain.invoke(
        {"messages": trimmed_messages, "language": state["language"]}
    )
    return {"messages": [response]}


workflow.add_edge(START, "llm")
workflow.add_node("llm", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

- Bây giờ nếu chúng ta thử hỏi tên của mô hình, nó sẽ không biết vì chúng ta đã cắt bớt phần đó trong lịch sử trò chuyện:

In [49]:
config = {"configurable": {"thread_id": "abc567"}}
query = "What is my name?"
language = "English"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I’m sorry, but I don’t know your name. What is it?


- Nhưng nếu chúng ta hỏi về thông tin nằm trong vài tin nhắn cuối cùng, nó sẽ ghi nhớ:

In [50]:
config = {"configurable": {"thread_id": "abc678"}}
query = "What math problem did I ask?"
language = "English"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

You asked what 2 + 2 equals.


### **Streaming**

- Bây giờ chúng ta đã có một chatbot hoạt động. Tuy nhiên, một cân nhắc thực sự quan trọng về UX đối với các ứng dụng chatbot là phát trực tuyến. LLM đôi khi có thể mất một thời gian để phản hồi, do đó, để cải thiện trải nghiệm người dùng, một điều mà hầu hết các ứng dụng thực hiện là phát trực tuyến lại từng mã thông báo khi nó được tạo. Điều này cho phép người dùng xem tiến trình.

- Thực ra, việc này cực kỳ dễ thực hiện!

- Theo mặc định, .stream trong ứng dụng LangGraph của chúng tôi, các bước ứng dụng sẽ được truyền phát - trong trường hợp này, là bước duy nhất của phản hồi mô hình. Thiết lập stream_mode="messages" cho phép chúng tôi truyền phát các mã thông báo đầu ra thay thế:

In [51]:
config = {"configurable": {"thread_id": "abc789"}}
query = "Hi I'm Todd, please tell me a joke."
language = "English"

input_messages = [HumanMessage(query)]
for chunk, metadata in app.stream(
    {"messages": input_messages, "language": language},
    config,
    stream_mode="messages",
):
    if isinstance(chunk, AIMessage):  # Filter to just model responses
        print(chunk.content, end="|")

|Hi| Todd|!| Here|’s| a| joke| for| you|:| 

|Why| don|’t| scientists| trust| atoms|?| 

|Because| they| make| up| everything|!||